### Extração de dados youtube- visualizacão de videos

**Bibliotecas:**

* pandas
* numpy
* BeautifulSoup
* matplotlib

In [39]:
 pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


**Imports de bliotecas para utilização:**

In [40]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import codecs

***Local do arquivo html***

In [41]:
local_html = '/home/mateus/Downloads/youtube/histórico/histórico-de-visualização.html'

**Instância de objeto da biblioteca BeautifulSoup**

In [42]:
def cria_objeto_soup(html_doc):
    fp = codecs.open(html_doc, 'r') 
    sp = BeautifulSoup(fp)
    return sp

In [47]:
def coleta_divs(nome_div,obj):
    mydivs = obj.findAll("div", {"class": nome_div})
    return  mydivs

In [49]:
nome_divs = 'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1'
soup = cria_objeto_soup(local_html)

divs = coleta_divs(nome_divs,soup)


In [94]:
vi = divs[0].a


<a href="https://www.youtube.com/watch?v=PD9Lk1YleWk">EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEIO NO PÂNICO | Pânico 2021 #94</a>

In [57]:
vi['href']. #url video

<a href="https://www.youtube.com/watch?v=PD9Lk1YleWk">EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEIO NO PÂNICO | Pânico 2021 #94</a>

In [58]:
 vi.text #txto do video

'EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEIO NO PÂNICO | Pânico 2021 #94'

In [66]:
prox_a = divs[0].find_all('a')

In [69]:
prox_a[1]['href'] #url do canal

'https://www.youtube.com/channel/UC2IEgRZcIuemAPJqXa2nHvQ'

In [70]:
prox_a[1].text #Nome canal

'Pânico Retrô'

In [77]:
date = divs[0]

In [86]:
date = str(date).split('<br/>')

In [89]:
t = len(date)

In [92]:
date = date[t-1].split('</div>')

In [93]:
date[0]

'25 de jul. de 2021 17:55:05 BRT'

In [ ]:
for dado in divs:
    

In [53]:
informacoes_visualizacaao_videos = {
                                    'canal':[],
                                    'url_canal':[],
                                    'nome_video':[],
                                    'url_video':[],
                                    'data_visualizacao':[]
}